In [ ]:
import datetime
import pandas as pd


# 고객 정보 관리 클래스
class CustomerInfo:
    def __init__(self, customer_id, csv_file_path):
        self.customer_id = customer_id
        self.csv_file_path = csv_file_path
        self.customer_info = self._get_customer_info()
    # csv 파일에서 고객정보 필터링하는 함수
    def _get_customer_info(self):
        try:
            # CSV 파일 읽기
            customer_data = pd.read_csv(self.csv_file_path, dtype={'연락처': object})

            # 고객 ID로 데이터 검색(필터링)
            customer_info = customer_data[customer_data['고객아이디'] == self.customer_id]

            # 정보가 존재한다면
            if not customer_info.empty:
                # 고객 정보를 딕셔너리로 반환
                return customer_info.to_dict(orient='records')[0]
            else:
                raise ValueError(f"고객 ID {self.customer_id}에 해당하는 정보를 찾을 수 없습니다.")
        except FileNotFoundError:
            raise FileNotFoundError(f"CSV 파일 {self.csv_file_path}이(가) 존재하지 않습니다.")

    # 특정 컬럼값을 반환하는 함수
    def get_attribute(self, attribute):
        if attribute in self.customer_info:
            return self.customer_info[attribute]
        else:
            raise KeyError(f"'{attribute}' 속성이 고객 정보에 존재하지 않습니다.")


# 생년월일을 기준으로 만 나이 계산하는 함수
def calculate_age(birth_date):

    # 생년월일 분리
    birthYear = int(birth_date[:4])
    birthMonth = int(birth_date[4:6])
    birthDay = int(birth_date[6:])

    # 오늘 날짜 가져오기
    today = datetime.date.today()
    thisYear = today.year
    thisMonth = today.month
    thisDay = today.day

    # 생일이 지났는지 확인
    if thisMonth > birthMonth or (thisMonth == birthMonth and thisDay >= birthDay):
        age = thisYear - birthYear
    else:
        age = thisYear - birthYear - 1

    return age


# 기초대사율과 활동대사량을 곱하여 하루 권장 식사량 계산
def calculate_bmr(weight, height, age, sex, exercise_score):

    # 기본 BMR 계산식
    bmr = (10 * weight) + (6.25 * height) - (5 * age)

    # 성별에 따른 조정
    if sex == 'F':  # 여성
        bmr -= 161
    elif sex == 'M':  # 남성
        bmr += 5
    else:
        raise ValueError("성별은 'M' 또는 'F'로 입력해야 합니다.")

    # 운동량에 따른 조정값 설정
    activity_multiplier = {
        'A': 1.9,    # 매우 높은 활동량
        'B': 1.725,  # 높은 활동량
        'C': 1.55,   # 보통 활동량
        'D': 1.375,  # 적은 활동량
        'E': 1.2     # 거의 활동하지 않음
    }

    # 유효한 운동량인지 확인
    if exercise_score not in activity_multiplier:
        raise ValueError("운동량은 'A', 'B', 'C', 'D', 'E' 중 하나여야 합니다.")

    # 운동량 곱하기 값 적용
    adjusted_bmr = bmr * activity_multiplier[exercise_score]
    return adjusted_bmr


# 실행
csv_file_path = 'FOOD_DB/food_project_user_info.csv'  # 고객 정보가 저장된 CSV 파일 경로
customer_id = input("고객 ID를 입력하세요: ")  # 고객 ID 입력

try:
    # 고객 정보 클래스 생성
    customer = CustomerInfo(customer_id, csv_file_path)

    # 고객 정보 가져오기
    birth_date = str(customer.get_attribute('생년월일'))
    weight = customer.get_attribute('몸무게')
    height = customer.get_attribute('키')
    sex = customer.get_attribute('성별')
    exercise_score = customer.get_attribute('운동량')
    name = customer.get_attribute('고객명')
    # 나이 계산
    age = calculate_age(birth_date)

    # BMR 계산
    adjusted_bmr_value = calculate_bmr(weight, height, age, sex, exercise_score)
    one_meal_value = adjusted_bmr_value / 3

    print(f"고객명: {name}님")
    print(f"나이: {age}세")         
    if sex == "M":
        print("성별: 남자")    
    else:
        print("성별: 여자")
    print(f"운동량을 고려한 조정된 BMR: {adjusted_bmr_value:.2f} kcal")
    print(f"한끼 권장 식사량: {one_meal_value:.2f} kcal")

except Exception as e:
    print(f"오류 발생: {e}")


고객명: 김태윤님
나이: 74세
성별: 남자
운동량을 고려한 조정된 BMR: 2596.44 kcal
한끼 권장 식사량: 865.48 kcal
